## 1. 데이터 분리와 모형 생성, 예측, 평가

  - **핵심 비유**: 학생의 진짜 실력은 풀어본 문제가 아닌, **처음 보는 문제**로 평가해야 함. 머신러닝 모델의 성능도 마찬가지임.
  - **일반화 (Generalization)**: 모델이 처음 보는 데이터에 대해 정확하게 예측하는 능력.
  - **과적합 (Overfitting)**: 모델이 학습 데이터의 패턴, 심지어 노이즈까지 전부 **암기**해버리는 현상.
      - **결과**: 학습 데이터에 대해서는 거의 100% 성능을 보이지만, 새로운 데이터에 대해서는 성능이 매우 저조함.
  - **해결 전략**: 모델의 진짜 **일반화 성능**을 공정하게 평가하기 위해, 전체 데이터를 학습용(Training Set)과 검증용(Test Set)으로 반드시 분리해야 함.

## 2. 핵심 원리 파헤치기 (Deep Dive)

  - 머신러닝 모델 개발의 **골든 룰**은 데이터를 두 그룹으로 나누어, 모델의 예측 결과와 실제 정답을 비교하는 것임.

  - **학습용 데이터셋 (Training Set)**

      - **목적**: 모델을 **학습**시키거나 **훈련**시키는 데 사용되는 데이터. (비유: 교과서, 문제집)
      - **역할**: 모델이 데이터의 통계적 관계, 패턴, 최적의 의사결정 경계(Decision Boundary)를 찾도록 함.
      - **비율**: 일반적으로 전체 데이터의 약 70~80%를 차지함.

  - **검증용 데이터셋 (Test Set)**

      - **목적**: 학습이 완료된 모델의 최종 **성능**을 평가하기 위한 데이터.
      - **규칙**: 모델 학습 과정에서 단 한 번도 사용되지 않은, **완전히 격리된 데이터**여야 함. (비유: 처음 보는 시험 문제)
      - **역할**: 모델의 **일반화 성능**, 즉 진짜 실력을 객관적으로 측정하는 척도로 사용됨.

  - **데이터 분할 비율과 편향-분산 트레이드오프**

      - 정해진 법칙은 없으며, 편향-분산 트레이드오프(Bias-Variance Tradeoff)와 관련 있음.
      - 학습 데이터가 너무 적으면 **높은 편향(과소적합)**, 검증 데이터가 너무 적으면 높은 분산(불안정한 평가)의 문제가 발생할 수 있음.
      - 경험적으로 **7:3**이나 **8:2** 비율이 널리 사용됨.

  - **층화추출법 (Stratified Sampling)**

      - **문제 상황**: **불균형 데이터(Imbalanced Data)**(예: 사기 거래 1%, 정상 거래 99%)를 무작위로 추출하면, 검증용 데이터에 소수 클래스가 포함되지 않을 수 있음.
      - **해결책**: 원본 데이터의 **클래스 비율**을 학습용과 검증용 데이터셋 모두에 **동일하게 유지**시켜주는 샘플링 기법.
      - **중요성**: 분류 문제에서 모델을 공정하게 평가하기 위한 필수적인 기법임.

## 3. 실전 코드 분석 및 적용 (Code Walkthrough & Application)

### 3.1. `random.sample()` 활용하기


  - Python의 기본 **`random`** 모듈을 사용하여 데이터의 **인덱스**를 직접 샘플링하는 기초적인 방법

In [1]:
import random
import seaborn as sns
import pandas as pd

# 재현성을 위해 난수 시드 설정 (언제 실행해도 같은 결과를 얻기 위함)
random.seed(10)

# iris 데이터셋 로드
iris = sns.load_dataset("iris")

# 150개 데이터 중 70%에 해당하는 105개의 인덱스를 무작위로 비복원 추출
inds = random.sample(population=range(150), k=int(150 * 0.7))
print(f"추출된 인덱스 개수: {len(inds)}")

# 추출된 인덱스 리스트를 기반으로 학습용 데이터셋 생성
train = iris.loc[inds, :]
train.sort_index(inplace=True) # 보기 쉽게 인덱스 기준으로 정렬

# 학습용 데이터셋에 포함되지 않은 나머지 데이터로 검증용 데이터셋 생성
# ~ 연산자는 NOT을 의미하며, isin()은 포함 여부를 확인합니다.
test = iris.loc[~iris.index.isin(train.index)]
test.sort_index(inplace=True) # 보기 쉽게 인덱스 기준으로 정렬

print("--- Train Dataset Head ---")
print(train.head())
print("\n--- Test Dataset Head ---")
print(test.head())

추출된 인덱스 개수: 105
--- Train Dataset Head ---
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
5           5.4          3.9           1.7          0.4  setosa
7           5.0          3.4           1.5          0.2  setosa

--- Test Dataset Head ---
    sepal_length  sepal_width  petal_length  petal_width species
1            4.9          3.0           1.4          0.2  setosa
2            4.7          3.2           1.3          0.2  setosa
6            4.6          3.4           1.4          0.3  setosa
13           4.3          3.0           1.1          0.1  setosa
16           5.4          3.9           1.3          0.4  setosa


##### 코드 해설

### 4.2. `pandas.DataFrame.sample()` 활용하기

  - Pandas 데이터프레임의 **`sample()`** 메소드를 활용하는 더 간편한 방법
  
    **`frac`** 인자에 비율을 지정함

In [2]:
import seaborn as sns
import pandas as pd

iris = sns.load_dataset("iris")

# frac=0.7로 전체 데이터의 70%를 무작위로 추출
# random_state는 재현성을 위한 시드값입니다.
train = iris.sample(frac=0.7, random_state=1)

# train 데이터의 인덱스를 제외한 나머지로 test 데이터 구성
test = iris.loc[~iris.index.isin(train.index)]

print("--- Train Dataset Head ---")
print(train.head())
print("\n--- Test Dataset Head ---")
print(test.head())

--- Train Dataset Head ---
     sepal_length  sepal_width  petal_length  petal_width     species
14            5.8          4.0           1.2          0.2      setosa
98            5.1          2.5           3.0          1.1  versicolor
75            6.6          3.0           4.4          1.4  versicolor
16            5.4          3.9           1.3          0.4      setosa
131           7.9          3.8           6.4          2.0   virginica

--- Test Dataset Head ---
   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
7           5.0          3.4           1.5          0.2  setosa
8           4.4          2.9           1.4          0.2  setosa


##### 코드 해설



### 4.3. `sklearn.model_selection.train_test_split()` 활용하기 (표준 방법)

  - **`scikit-learn`** 라이브러리의 **`train_test_split`** 함수는 데이터 분리를 위한 가장 **보편적이고 표준적인 방법**임.

In [3]:
import seaborn as sns
from sklearn.model_selection import train_test_split

iris = sns.load_dataset("iris")

# 독립변수(X)와 종속변수(y) 분리
X = iris.iloc[:, :-1] # 마지막 'species' 열을 제외한 모든 특성(feature) 열
y = iris.iloc[:, -1]  # 마지막 'species' 열 (타겟, target)

# 데이터를 학습용과 검증용으로 분리
# test_size=0.3 으로 검증용 데이터 비율을 30%로 지정
# random_state=1 로 결과를 고정
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

# 나눠진 데이터셋들의 크기(shape) 확인
print(f"train_X shape: {train_X.shape}") # 학습용 독립변수
print(f"test_X shape: {test_X.shape}")   # 검증용 독립변수
print(f"train_y shape: {train_y.shape}") # 학습용 종속변수
print(f"test_y shape: {test_y.shape}")   # 검증용 종속변수

print("\n--- Test y value counts (Random) ---")
print(test_y.value_counts())

train_X shape: (105, 4)
test_X shape: (45, 4)
train_y shape: (105,)
test_y shape: (45,)

--- Test y value counts (Random) ---
species
versicolor    18
setosa        14
virginica     13
Name: count, dtype: int64


##### 코드 해설



### 4.4. 층화추출법 적용하기


  - **`train_test_split`** 함수의 **`stratify`** 옵션을 사용하면 클래스 비율을 유지하며 분할 가능함.

In [4]:
import seaborn as sns
from sklearn.model_selection import train_test_split

iris = sns.load_dataset("iris")
X = iris.iloc[:, :-1]
y = iris.species # .iloc[:, -1] 과 동일한 표현입니다.

# stratify=y 옵션을 추가하여 y(species)의 클래스 비율을 유지하며 분리
train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.3,
                                                    random_state=42)

print("--- Test y value counts (Stratified) ---")
print(test_y.value_counts())

--- Test y value counts (Stratified) ---
species
virginica     15
versicolor    15
setosa        15
Name: count, dtype: int64


##### 코드 해설

### 5. 모형 생성, 예측, 평가

  - 분리된 데이터를 사용하여 실제 머신러닝 모델을 만들고 평가하는 전체 과정을 **의사결정나무**와 **인공신경망** 모델을 예시로 살펴봄.

In [5]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# 1. 데이터 준비 및 분리 (층화추출법 적용)
iris = sns.load_dataset("iris")
X = iris.iloc[:, :-1]
y = iris.iloc[:, -1]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# 2. 모형 생성 및 학습 (fit)
# scikit-learn의 모든 모델은 fit() 메소드로 학습합니다.
print("모델 학습을 시작합니다...")
# 의사결정나무 모델 생성 및 학습
dt_model = DecisionTreeClassifier(random_state=1)
dt_model.fit(train_X, train_y)

# 인공신경망 모델 생성 및 학습
mlp_model = MLPClassifier(hidden_layer_sizes=(50,50,20), max_iter=500, random_state=1, activation='relu')
mlp_model.fit(train_X, train_y)
print("모델 학습이 완료되었습니다.")

# 3. 예측 (predict)
# 학습된 모델을 사용해 '처음 보는 데이터'인 test_X의 결과를 예측합니다.
dt_pred_y = dt_model.predict(test_X)
mlp_pred_y = mlp_model.predict(test_X)

# 4. 모형 평가 (score 및 crosstab)
# score() 함수는 정확도(accuracy)를 반환합니다. (전체 예측 중 맞은 것의 비율)
dt_accuracy = dt_model.score(test_X, test_y)
mlp_accuracy = mlp_model.score(test_X, test_y)

print(f"\nDecision Tree Accuracy: {dt_accuracy:.4f}")
print(f"MLP Classifier Accuracy: {mlp_accuracy:.4f}")

# crosstab으로 실제값(index)과 예측값(columns)을 비교하는 오차 행렬(Confusion Matrix) 생성
print("\n--- Decision Tree Confusion Matrix ---")
print(pd.crosstab(test_y, dt_pred_y, rownames=['Actual'], colnames=['Predicted']))

print("\n--- MLP Classifier Confusion Matrix ---")
print(pd.crosstab(test_y, mlp_pred_y, rownames=['Actual'], colnames=['Predicted']))

모델 학습을 시작합니다...
모델 학습이 완료되었습니다.

Decision Tree Accuracy: 0.9778
MLP Classifier Accuracy: 1.0000

--- Decision Tree Confusion Matrix ---
Predicted   setosa  versicolor  virginica
Actual                                   
setosa          15           0          0
versicolor       0          15          0
virginica        0           1         14

--- MLP Classifier Confusion Matrix ---
Predicted   setosa  versicolor  virginica
Actual                                   
setosa          15           0          0
versicolor       0          15          0
virginica        0           0         15


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


##### 코드 해설

## 5. 핵심 요약 (Key Takeaways)


  - **머신러닝 평가의 황금률**: **훈련 데이터로 테스트하지 말 것**. **일반화 성능**을 측정하기 위해 데이터 분리는 필수이며, 이를 어기면 **과적합**된 모델을 배포하는 실수를 범하게 됨.
  - **`train_test_split`이 표준인 이유**: 사용이 간편하고 `random_state`로 **재현성** 보장, `stratify`로 **안정성** 확보가 가능함. Scikit-learn 생태계와 호환성이 높아 가장 표준적인 도구임.
  - **신뢰할 수 있는 분류 모델을 위해 `stratify`를 기억하세요**: 특히 **클래스 불균형** 데이터에서 `stratify` 옵션은 원본 데이터의 클래스 분포를 유지시켜, 모델 평가의 **신뢰도를 극적으로 높여줌**.